In [1]:
%run initialising.ipynb

In [2]:
def formatP(p):
    if p >= 0.05:
        return str(round(p, 3))
    elif p >= 0.001:
        return str(round(p, 3)) + '*'
    else:
        return '<0.001*'

In [3]:
def compare_ex30_vs_ld30(measure_type, threshold):
    '''Prints statistical analysis comparing %acc of EX30 vs LD30 for a
    given measure_type and minimum expansion threshold'''
    
    if threshold:
        print("Is the %%accuracy of EX30 significantly different to LD30, when requested expansion > %s mm?\n" % threshold)
    else:
        print("Is the %accuracy of EX30 significantly different to LD30, for any level of requested expansion?")

    all_ex30 = pd.Series([])
    all_ld30 = pd.Series([])
    
    newDf = pd.DataFrame(columns=['Tooth Type', 'LD30 %Accuracy', 'SD1', 'n1', 'EX30 %Accuracy', 'SD2', 'n2', 'p'])
    
    for i, j in zip([1, 3], [2, 4]):
        for toothNum in range(3, 8):
            if measure_type == '.cc' and toothNum == 7:
                continue

            antimere = '%s%s-%s%s' % (i, toothNum, j, toothNum)
            full_antimere = '%s%s-%s%s%s' % (i, toothNum, j, toothNum, measure_type)

            col1 = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'EX30')            
            col2 = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'LD30')
            stat, p = stats.ttest_ind(col1, col2, nan_policy='omit')

            print('---%s---' % full_antimere)
            print('EX30: mean=%s\tstd=%s\tn=%s' % (col1.mean(), col1.std(), col1.count()))
            print('LD30: mean=%s\tstd=%s\tn=%s' % (col2.mean(), col2.std(), col2.count()))
            s = 'p=%s' % p
            if p < 0.05:
                s = '*' + s
            print('%s\n' % s)
            
            all_ex30 = all_ex30.append(col1)
            all_ld30 = all_ld30.append(col2)
            
            # add row to table (newDf)
            k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1       
            newDf.loc[k] = [antimere_dict[antimere], col2.mean(), col2.std(), col2.count(), col1.mean(), col1.std(), col1.count(), formatP(p)]
    
    # calculate 'overall' statistic
    stat, p = stats.ttest_ind(all_ex30, all_ld30, nan_policy='omit')
    print('ALL EX30%s: mean=%s\tstd=%s\tn=%s' % (measure_type, all_ex30.mean(), all_ex30.std(), all_ex30.count()))
    print('ALL LD30%s: mean=%s\tstd=%s\tn=%s' % (measure_type, all_ld30.mean(), all_ld30.std(), all_ld30.count()))
    
    s = 'p=%s' % p
    if p < 0.05:
        s = '*' + s
    print('%s\n\n\n' % s)
    
    # add 'overall' statistic to df
    k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1       
    newDf.loc[k] = ['Total', all_ld30.mean(), all_ld30.std(), all_ld30.count(), all_ex30.mean(), all_ex30.std(), all_ex30.count(), formatP(p)]
    newDf = newDf.round({'LD30 %Accuracy': 1,
                 'SD1': 1,
                 'n1': 0,
                 'EX30 %Accuracy': 1,
                 'SD2': 1,
                 'n2': 0
    })
    newDf.to_csv(os.path.join('output/tables/ex30-vs-ld30-%s-%s.csv'%(measure_type, threshold)), index=False)
    
    return

In [4]:
def compare_predicted_vs_achieved(material, measure_type, threshold):
    '''Prints statistical analysis comparing %acc of  vs LD30 for a
    given measure_type and minimum expansion threshold'''
    
    assert(material in ['LD30', 'EX30'])
    
    if threshold:
        print("Is the actual expansion of %s significantly different to its predicted expansion, for cases where predicted expansion >%s mm?\n"\
          % (material, threshold))
    else:
        print("Is the actual expansion of %s significantly different to its predicted expansion, for any level of predicted expansion?\n"\
          % (material))
    
    # create new df which will serve as table of results
    newDf = pd.DataFrame(columns=['Tooth Type', 'n', 'Predicted Expansion', 'SD1', 'Achieved Expansion', 'SD2', 'p', '%Accuracy', 'SD3'])
    
    for i, j in zip([1, 3], [2, 4]):
        for toothNum in range(3, 8):
            if measure_type == '.cc' and toothNum == 7:
                continue

            antimere = '%s%s-%s%s' % (i, toothNum, j, toothNum)
            full_antimere = '%s%s-%s%s%s' % (i, toothNum, j, toothNum, measure_type)

            col1 = getColumn(df, 'aΔ%s%s' % (antimere, measure_type), antimere, threshold, measure_type, material)
            col2 = getColumn(df, 'pΔ%s%s' % (antimere, measure_type), antimere, threshold, measure_type, material)
            stat, p = stats.ttest_rel(col1, col2, nan_policy='omit')

            print('---%s---' % full_antimere)
            print('FΔ: mean=%s\tstd=%s\tn=%s' % (col1.mean(), col1.std(), col1.count()))
            print('PΔ: mean=%s\tstd=%s\tn=%s' % (col2.mean(), col2.std(), col2.count()))
            
            s = 'p=%s' % p
            if p < 0.05:
                s = '*' + s
            print('%s\n' % (s))
            
            acc = getColumn(df, '%s %%acc' % full_antimere, antimere, threshold, measure_type, 'LD30')
            
            ## add row to table (newDf)
            k = 0 if pd.isnull(newDf.index.max()) else newDf.index.max()+1
            n = min([col1.count(), col2.count()])
            newDf.loc[k] = [antimere_dict[antimere], n, col2.mean(), col2.std(), col1.mean(), col1.std(), formatP(p), acc.mean(), acc.std()]
    newDf = newDf.round({'n':0,
                 'Predicted Expansion':2,
                 'SD1':2,
                 'Achieved Expansion': 2,
                 'SD2': 2,
                 '%Accuracy':1,
                 'SD3':1
    })
    newDf.to_csv(os.path.join('output/tables/%s-%s-%s-acc.csv'%(material, measure_type, threshold)), index=False)
    return

In [5]:
for measure_type in MEASURE_TYPES:
    compare_ex30_vs_ld30(measure_type, THRESHOLD)
    compare_ex30_vs_ld30(measure_type, False)

Is the %accuracy of EX30 significantly different to LD30, when requested expansion > 2.0 mm?

---13-23.cc---
EX30: mean=79.84874014998911	std=20.16718342688374	n=25
LD30: mean=81.08505378868696	std=16.938318120384668	n=37
p=0.795008909149

---14-24.cc---
EX30: mean=78.83147465215973	std=24.704532886179564	n=32
LD30: mean=83.81808635885535	std=16.857322335132025	n=56
p=0.264688741034

---15-25.cc---
EX30: mean=72.29305098810151	std=19.68300866199299	n=31
LD30: mean=82.2799839304543	std=18.109856745694557	n=52
*p=0.0210686693129

---16-26.cc---
EX30: mean=49.44188309094142	std=28.49436866792993	n=16
LD30: mean=82.89815952209653	std=28.547151249406443	n=31
*p=0.000419559183806

---33-43.cc---
EX30: mean=83.25618203202332	std=17.408604783556267	n=22
LD30: mean=91.4686472113691	std=17.89968229546734	n=40
p=0.0860831541951

---34-44.cc---
EX30: mean=77.65496112092524	std=35.61500300187484	n=26
LD30: mean=92.10691447474062	std=28.44587325624825	n=47
p=0.0618073099445

---35-45.cc---
EX30: mea

LD30: mean=91.76941403718433	std=27.77108544554979	n=26
*p=0.00989011347271

---37-47.gm---
EX30: mean=37.0012616721903	std=36.61950828102458	n=5
LD30: mean=63.14404172718276	std=46.237014968211	n=11
p=0.286113561658

ALL EX30.gm: mean=59.8813780801604	std=27.31564228822323	n=173
ALL LD30.gm: mean=76.76526954996089	std=26.35159224201929	n=310
*p=7.3483273868e-11



Is the %accuracy of EX30 significantly different to LD30, for any level of requested expansion?
---13-23.gm---
EX30: mean=55.87769505817063	std=69.5127591085436	n=35
LD30: mean=95.30480040764267	std=234.79853942259916	n=60
p=0.336011582737

---14-24.gm---
EX30: mean=64.30890967736993	std=18.04346874086149	n=35
LD30: mean=77.22826381662459	std=22.080924072485992	n=60
*p=0.00420303038306

---15-25.gm---
EX30: mean=82.41594855950153	std=123.22219108539146	n=35
LD30: mean=80.25002547820914	std=30.966235986444524	n=60
p=0.897040843458

---16-26.gm---
EX30: mean=-12.31751331645456	std=273.5842597876729	n=35
LD30: mean=92.676686428

In [6]:
for measure_type in MEASURE_TYPES:
    compare_predicted_vs_achieved('LD30', measure_type, THRESHOLD)
    compare_predicted_vs_achieved('LD30', measure_type, False)

Is the actual expansion of LD30 significantly different to its predicted expansion, for cases where predicted expansion >2.0 mm?

---13-23.cc---
FΔ: mean=2.767567567567567	std=1.046967593166637	n=37
PΔ: mean=3.4351351351351336	std=1.133876393473149	n=37
*p=6.96213408974e-08

---14-24.cc---
FΔ: mean=3.3875000000000006	std=1.2482806356963745	n=56
PΔ: mean=4.039285714285715	std=1.2466214080726081	n=56
*p=1.55682349418e-09

---15-25.cc---
FΔ: mean=3.0807692307692305	std=1.1226786004101998	n=52
PΔ: mean=3.7653846153846158	std=1.1812065309581894	n=52
*p=1.69228331643e-08

---16-26.cc---
FΔ: mean=2.4612903225806453	std=0.7952682647404116	n=31
PΔ: mean=3.0612903225806445	std=0.6427427786991402	n=31
*p=0.0011667075026

---33-43.cc---
FΔ: mean=3.1125	std=0.9285189475166544	n=40
PΔ: mean=3.46	std=0.9875636947974803	n=40
*p=0.00233544219987

---34-44.cc---
FΔ: mean=3.725531914893618	std=1.5437768600171835	n=47
PΔ: mean=4.004255319148936	std=1.1684172584812087	n=47
p=0.0533728746412

---35-45.cc---

*p=0.00206283408492

---17-27.ct---
FΔ: mean=2.34375	std=1.0513520206993332	n=8
PΔ: mean=3.4974999999999996	std=0.7130768341690608	n=8
*p=0.0406843244064

---33-43.ct---
FΔ: mean=2.924390243902439	std=1.0431731610343693	n=41
PΔ: mean=3.1773170731707308	std=0.9332819091524375	n=41
*p=0.0159926400396

---34-44.ct---
FΔ: mean=3.688260869565218	std=1.4946234075628495	n=46
PΔ: mean=4.117173913043479	std=1.1878265175305665	n=46
*p=0.000665990465667

---35-45.ct---
FΔ: mean=4.094363636363636	std=1.7700405557567236	n=55
PΔ: mean=4.485636363636364	std=1.766121261342015	n=55
*p=0.0068214482362

---36-46.ct---
FΔ: mean=3.3727777777777783	std=1.3796741046060967	n=36
PΔ: mean=3.7316666666666674	std=1.358634819010823	n=36
p=0.105628209524

---37-47.ct---
FΔ: mean=2.5625	std=1.2600608450917494	n=16
PΔ: mean=3.4074999999999993	std=0.9339057768319037	n=16
*p=0.0206554567116

           Tooth Type   n  Predicted Expansion   SD1  Achieved Expansion  \
0        Upper Canine  36                 3.52  1.23 